In [ ]:
#default_exp model.alexnet

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from collections import namedtuple
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Callable, Any, Optional, Tuple, List
import numpy as np
from torchsummary import summary
from fastai.basics import *

/home/mayrajeo/miniconda3/envs/enveco/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# AlexNet -style CNN for voxel data

> Process 40x40x105 voxels with traditional CNN

In [ ]:
#export

@delegates(Learner.__init__)
def alexnetvoxel_learner(dls, loss_func=None, y_range=None, config=None, n_out=None, n_in=3, **kwargs):
    
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    model = AlexNetVoxel(n_in=n_in, num_classes=n_out)
    apply_init(model, nn.init.kaiming_normal_)
    learn = Learner(dls, model, loss_func=loss_func, **kwargs)
    return learn

## Building blocks and utils

In [ ]:
# export
    
def calc_same_padding_2d(inshape:tuple, kernel:tuple, strides:tuple) -> Tuple[int, int]:
    """
    Calculate layer sizes similarly to tensorflow padding='same' for 2d data.
    [left, right, top, bot] is the order for F.pad.
    Has some kind of performance penalty.
    """
    _, _, in_h, in_w = inshape
    krl_h, krl_w = kernel
    str_h, str_w = strides
    
    out_h = np.ceil(float(in_h) / float(str_h))
    out_w = np.ceil(float(in_w) / float(str_w))
        
    # width padding
    if (in_w % str_w == 0):
        pad_along_w = max(krl_w - str_w, 0)
    else:
        pad_along_w = max(krl_w - (in_w % str_w), 0)
        
    # height padding
    if (in_h % str_h == 0):
        pad_along_h = max(krl_h - str_h, 0)
    else:
        pad_along_h = max(krl_h - (in_h % str_h), 0)
        
    pad_left = pad_along_w // 2
    pad_right = pad_along_w - pad_left
    pad_top = pad_along_h // 2
    pad_bot = pad_along_h - pad_top
    return (pad_left, pad_right, pad_top, pad_bot)

In [ ]:
# export
        
class PaddedConv2d(nn.Module):
    "Module for Conv3d-BN-relu, with the option for tensorflow-style `same` padding"
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias:bool=False,
        same_padding:bool=False,
        **kwargs: Any
    ) -> None:
        super(PaddedConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=bias, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
        self.same_padding = same_padding
        self.kernel = kwargs['kernel_size']
        self.stride = kwargs['stride']
        self.padding_size = None

    def forward(self, x: Tensor) -> Tensor:
        if self.same_padding:
            if self.padding_size == None:
                self.padding_size = calc_same_padding_2d(x.shape, self.kernel, self.stride)
            x = F.pad(x, self.padding_size)
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class PaddedMaxPool2d(nn.Module):
    "Module for MaxPool3d with optional tensorflow-style `same` padding"
    
    def __init__(self, same_padding:bool=False, **kwargs: Any) -> None:
        super(PaddedMaxPool2d, self).__init__()
        self.pool = nn.MaxPool2d(**kwargs)
        self.same_padding = same_padding
        self.kernel = kwargs['kernel_size']
        self.stride = kwargs['stride']
        self.padding_size = None
        
    def forward(self, x:Tensor) -> Tensor:
        if self.same_padding:
            if self.padding_size == None:
                self.padding_size = calc_same_padding_2d(x.shape, self.kernel, self.stride)
            x = F.pad(x, self.padding_size)
        x = self.pool(x)
        return x

## AlexnetModel

In [ ]:
#export

class AlexNetVoxel(nn.Module):
    "Custom AlexNet for voxel data"
    
    def __init__(self, n_in:int, num_classes:int=1) -> None:
        super(AlexNetVoxel, self).__init__()
        
        self.conv1 = PaddedConv2d(n_in, 1050, kernel_size=(2,2), stride=(1,1), same_padding=True)
        self.pool1 = PaddedMaxPool2d(kernel_size=(2,2), stride=(2,2), same_padding=False)
        
        self.conv2 = PaddedConv2d(1050, 256, kernel_size=(3,3), stride=(1,1), same_padding=True)
        self.pool2 = PaddedMaxPool2d(kernel_size=(2,2), stride=(2,2), same_padding=False)
        
        self.conv3 = PaddedConv2d(256, 384, kernel_size=(3,3), stride=(1,1), same_padding=True)
        self.conv4 = PaddedConv2d(384, 384, kernel_size=(3,3), stride=(1,1), same_padding=True)
        self.pool4 = PaddedMaxPool2d(kernel_size=(2,2), stride=(2,2), same_padding=False)
        
        self.conv5 = PaddedConv2d(384, 256, kernel_size=(3,3), stride=(1,1), same_padding=True)
        self.pool5 = PaddedMaxPool2d(kernel_size=(2,2), stride=(2,2), same_padding=False)
        
        self.flat = nn.Flatten()
        self.dense1 = LinBnDrop(1024,4096)
        self.dense2 = LinBnDrop(4096,4096)
        self.dense3 = LinBnDrop(4096,1000)
        self.output = LinBnDrop(1000,num_classes, bn=False)
        
    def forward(self, x:Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool4(x)
        x = self.conv5(x)
        x = self.pool5(x)
        x = self.flat(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.output(x)
        return x

In [ ]:
from torchsummary import summary

In [ ]:
model = AlexNetVoxel(n_in=105)

In [ ]:
summary(model, (105,40,40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 1050, 40, 40]         441,000
       BatchNorm2d-2         [-1, 1050, 40, 40]           2,100
      PaddedConv2d-3         [-1, 1050, 40, 40]               0
         MaxPool2d-4         [-1, 1050, 20, 20]               0
   PaddedMaxPool2d-5         [-1, 1050, 20, 20]               0
            Conv2d-6          [-1, 256, 20, 20]       2,419,200
       BatchNorm2d-7          [-1, 256, 20, 20]             512
      PaddedConv2d-8          [-1, 256, 20, 20]               0
         MaxPool2d-9          [-1, 256, 10, 10]               0
  PaddedMaxPool2d-10          [-1, 256, 10, 10]               0
           Conv2d-11          [-1, 384, 10, 10]         884,736
      BatchNorm2d-12          [-1, 384, 10, 10]             768
     PaddedConv2d-13          [-1, 384, 10, 10]               0
           Conv2d-14          [-1, 384,

In [ ]:
# hide

from nbdev.export import notebook2script
notebook2script()
!nbdev_build_docs

Converted 00_data.image.ipynb.
Converted 01_data.las.ipynb.
Converted 02_tabular.preprocessing.ipynb.
Converted 03_model.inception3dv3.ipynb.
Converted 04_interpretation.ipynb.
Converted 05_metrics.ipynb.
Converted 06_model.ensemble.ipynb.
Converted 07_model.alexnet.ipynb.
Converted index.ipynb.
converting: /scratch/mayrajan/enveco/nbs/07_model.alexnet.ipynb
converting /scratch/mayrajan/enveco/nbs/index.ipynb to README.md
